In [1]:
import os
import numpy as np
import random
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation
from sklearn.model_selection import train_test_split

In [2]:
batchsize =8

In [3]:
np.random.seed(42)
random.seed(42)

In [4]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'F:\4-1\AIML PROJECT\mrleyedataset\new short train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

Found 3677 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'F:\4-1\AIML PROJECT\mrleyedataset\new test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 1946 images belonging to 2 classes.


In [6]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [7]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(x= train_data,y=None,batch_size = 8, epochs=10)

model.save(r'F:\4-1\AIML PROJECT\models\model.h5')


Epoch 1/10


460/460 [==============================] - 61s 116ms/step - loss: 0.3036 - accuracy: 0.8790
Epoch 2/10
460/460 [==============================] - 27s 59ms/step - loss: 0.2484 - accuracy: 0.9016
Epoch 3/10
460/460 [==============================] - 28s 61ms/step - loss: 0.2287 - accuracy: 0.9100
Epoch 4/10
460/460 [==============================] - 33s 71ms/step - loss: 0.2176 - accuracy: 0.9113
Epoch 5/10
460/460 [==============================] - 29s 62ms/step - loss: 0.1946 - accuracy: 0.9239
Epoch 6/10
460/460 [==============================] - 29s 63ms/step - loss: 0.1982 - accuracy: 0.9171
Epoch 7/10
460/460 [==============================] - 29s 63ms/step - loss: 0.2000 - accuracy: 0.9260
Epoch 8/10
460/460 [==============================] - 29s 63ms/step - loss: 0.1981 - accuracy: 0.9173
Epoch 9/10
460/460 [==============================] - 29s 63ms/step - loss: 0.1979 - accuracy: 0.9192
Epoch 10/10
460/460 [==============================] - 29s 63ms/step - loss: 0

F:\4-1\jpn\project\my_venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
#giving true labels to close eyes as '0'

# Specify the path to your image folder
image_folder = r'F:\4-1\AIML PROJECT\mrleyedataset\new test\close eyes'

# List all files in the image folder
all_images = os.listdir(image_folder)

# Assign the same label (e.g., 1) to all images
label = 0

# Create a list of labels corresponding to each image
labels = [label] * len(all_images)

import pandas as pd

# Create a DataFrame
df = pd.DataFrame({'ImageID': all_images, 'Label': labels})
df.head()

,ImageID,Label
0,s0001_00001_0_0_0_0_0_01.png,0
1,s0001_00002_0_0_0_0_0_01.png,0
2,s0001_00003_0_0_0_0_0_01.png,0
3,s0001_00004_0_0_0_0_0_01.png,0
4,s0001_00005_0_0_0_0_0_01.png,0


In [10]:
#giving true labels to open eyes as '1'

import os

# Specify the path to your image folder
image_folder1 = r'F:\4-1\AIML PROJECT\mrleyedataset\new test\open eyes'

# List all files in the image folder
all_images1 = os.listdir(image_folder1)

# Assign the same label (e.g., 1) to all images
label = 1

# Create a list of labels corresponding to each image
labels2 = [label] * len(all_images)

# Create a DataFrame
df2 = pd.DataFrame({'ImageID': all_images1, 'Label': labels2})
df2.head()

,ImageID,Label
0,s0001_01842_0_0_1_0_0_01.png,1
1,s0001_01843_0_0_1_0_0_01.png,1
2,s0001_01844_0_0_1_0_0_01.png,1
3,s0001_01845_0_0_1_0_0_01.png,1
4,s0001_01846_0_0_1_0_0_01.png,1


In [11]:
result_df = pd.concat([df, df2], ignore_index=True)
result_df.head()
#result_df.tail()

,ImageID,Label
0,s0001_00001_0_0_0_0_0_01.png,0
1,s0001_00002_0_0_0_0_0_01.png,0
2,s0001_00003_0_0_0_0_0_01.png,0
3,s0001_00004_0_0_0_0_0_01.png,0
4,s0001_00005_0_0_0_0_0_01.png,0


In [12]:
#predictions on test open eyes dataset and saving them under a dataframe 

import os
import cv2

import pandas as pd
from keras.preprocessing import image
from keras.models import load_model

# Load the pre-trained model
model = load_model(r'F:\4-1\AIML PROJECT\models\model.h5')

# Specify the path to your image folder
image_folder3 = r'F:\4-1\AIML PROJECT\mrleyedataset\new test\open eyes'

# Create lists to store filenames and predictions
image_filenames = []
predictions = []


for filename in os.listdir(image_folder3):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
        # Load and preprocess the image
        img_path = os.path.join(image_folder3, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (80, 80))  # Adjust size based on your model's input size
        img = img / 255.0  # Normalize pixel values to [0, 1]
        img = np.reshape(img, (1, 80, 80, 3))  # Adjust shape based on your model's input shape
        predictions1 = model.predict(img)
        predicted_class = np.argmax(predictions1)
        image_filenames.append(filename)
        predictions.append(predicted_class) 

# Create a DataFrame with image filenames and predictions
df3 = pd.DataFrame({'ImageID': image_filenames, 'Prediction': predictions})



1/1 [==============================] - 0s 50ms/step


In [13]:
# Display the DataFrame
print(df3.head())

                        ImageID  Prediction
0  s0001_01842_0_0_1_0_0_01.png           0
1  s0001_01843_0_0_1_0_0_01.png           1
2  s0001_01844_0_0_1_0_0_01.png           0
3  s0001_01845_0_0_1_0_0_01.png           1
4  s0001_01846_0_0_1_0_0_01.png           1


In [14]:
#predictions on test closed eyes dataset and saving them under a dataframe

import os
import cv2
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.models import load_model

# Load the pre-trained model
model = load_model(r'F:\4-1\AIML PROJECT\models\model.h5')

# Specify the path to your image folder
image_folder4 = r'F:\4-1\AIML PROJECT\mrleyedataset\new test\close eyes'

# Create lists to store filenames and predictions
image_filenames2 = []
predictions2 = []


for filename in os.listdir(image_folder4):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
        # Load and preprocess the image
        img_path2 = os.path.join(image_folder4, filename)
        img1 = cv2.imread(img_path2)
        img1 = cv2.resize(img1, (80, 80))  # Adjust size based on your model's input size
        img1 = img1 / 255.0  # Normalize pixel values to [0, 1]
        img1 = np.reshape(img1, (1, 80, 80, 3))  # Adjust shape based on your model's input shape
        predictions3 = model.predict(img1)
        predicted_class2 = np.argmax(predictions3)
        image_filenames2.append(filename)
        predictions2.append(predicted_class2) 
         


# Create a DataFrame with image filenames and predictions
df4 = pd.DataFrame({'ImageID': image_filenames2, 'Prediction': predictions2})



1/1 [==============================] - 0s 51ms/step


In [15]:
# Display the DataFrame
print(df4.head())


                        ImageID  Prediction
0  s0001_00001_0_0_0_0_0_01.png           0
1  s0001_00002_0_0_0_0_0_01.png           0
2  s0001_00003_0_0_0_0_0_01.png           0
3  s0001_00004_0_0_0_0_0_01.png           0
4  s0001_00005_0_0_0_0_0_01.png           0


In [16]:
predict_df = pd.concat([df4, df3], ignore_index=True)
predict_df.head()
#result_df.tail()

,ImageID,Prediction
0,s0001_00001_0_0_0_0_0_01.png,0
1,s0001_00002_0_0_0_0_0_01.png,0
2,s0001_00003_0_0_0_0_0_01.png,0
3,s0001_00004_0_0_0_0_0_01.png,0
4,s0001_00005_0_0_0_0_0_01.png,0


In [17]:
import pandas as pd
from sklearn.metrics import accuracy_score


# Merge DataFrames on the common ID column
df_merged = pd.merge(result_df, predict_df, on='ImageID')

# Extract true and predicted labels
y_truest = df_merged['Label']
y_predict = df_merged['Prediction']



accuracy = accuracy_score(y_truest, y_predict)
print(f"Accuracy: {accuracy}")




Accuracy: 0.8869475847893115


In [18]:
from sklearn.metrics import  classification_report
classification_report_result = classification_report(y_truest, y_predict)
print("Classification Report:\n", classification_report_result)

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.98      0.90       973
           1       0.97      0.80      0.88       973

    accuracy                           0.89      1946
   macro avg       0.90      0.89      0.89      1946
weighted avg       0.90      0.89      0.89      1946

